In [1]:
%matplotlib qt

import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
from poisson_fem import PoissonFEM
import ROM
import GenerativeSurrogate as gs
import Data as dta
import numpy as np
import scipy.sparse as sps
import scipy.sparse.linalg as lg
import time
import petsc4py
import sys
petsc4py.init(sys.argv)
from petsc4py import PETSc
import torch
from torch import optim

/home/constantin/anaconda3/envs/genDRROM/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/constantin/anaconda3/envs/genDRROM/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/constantin/anaconda3/envs/genDRROM/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/c

In [2]:
# Some parameters
lin_dim_rom = 4                      # Linear number of rom elements
a = np.array([1, 1, 0])              # Boundary condition function coefficients
dtype = torch.float                  # Tensor data type
supervised_samples = {n for n in range(16)}
unsupervised_samples = {n for n in range(16, 16)}

In [3]:
# Define mesh and boundary conditions
mesh = PoissonFEM.RectangularMesh(np.ones(lin_dim_rom)/lin_dim_rom)
# mesh.plot()

def origin(x):
    return np.abs(x[0]) < np.finfo(float).eps and np.abs(x[1]) < np.finfo(float).eps

def ess_boundary_fun(x):
    return 0.0
mesh.set_essential_boundary(origin, ess_boundary_fun)

def domain_boundary(x):
    # unit square
    return np.abs(x[0]) < np.finfo(float).eps or np.abs(x[1]) < np.finfo(float).eps or \
            np.abs(x[0]) > 1.0 - np.finfo(float).eps or np.abs(x[1]) > 1.0 - np.finfo(float).eps
mesh.set_natural_boundary(domain_boundary)

def flux(x):
    q = np.array([a[0] + a[2]*x[1], a[1] + a[2]*x[0]])
    return q

In [4]:
#Specify right hand side and stiffness matrix
rhs = PoissonFEM.RightHandSide(mesh)
rhs.set_natural_rhs(mesh, flux)
K = PoissonFEM.StiffnessMatrix(mesh)
rhs.set_rhs_stencil(mesh, K)

In [5]:
trainingData = dta.StokesData(supervised_samples, unsupervised_samples)
trainingData.read_data()
# trainingData.plotMicrostruct(1)
trainingData.reshape_microstructure_image()

In [6]:
# define rom
rom = ROM.ROM(mesh, K, rhs, trainingData.output_resolution**2)

In [7]:
model = gs.GenerativeSurrogate(rom, trainingData)
# model = gs.GenerativeSurrogate()
# model.load()

In [ ]:
# optimize lambdac first
# lambdac_iterations = 3e4
# for k in range(model.data.n_supervised_samples):
#     print('sample = ', k)
#     lr_init = 1e-2
#     model.lambdacOpt.param_groups[0]['lr'] = lr_init
#     lambdac_iter = 0
#     while lambdac_iter < lambdac_iterations and model.lambdacOpt.param_groups[0]['lr'] > 1e-4 * lr_init:
#         model.lambdac_step(k, lambdac_iter)
#         lambdac_iter += 1
for n in range(model.data.n_supervised_samples):
    print('sample == ', n)
    model.log_lambdac_mean[n].max_iter = 3e4
    model.log_lambdac_mean[n].converge(model, model.data.n_supervised_samples, mode=n)

sample ==  0
loss_lambda_c =  45945471369216.0
loss_lambda_c =  4139495194624.0
loss_lambda_c =  923278901248.0
loss_lambda_c =  561463427072.0
loss_lambda_c =  448313819136.0
loss_lambda_c =  411477934080.0
loss_lambda_c =  395287527424.0
loss_lambda_c =  385235648512.0
loss_lambda_c =  378070237184.0
loss_lambda_c =  372650147840.0
loss_lambda_c =  368317300736.0
loss_lambda_c =  364676907008.0
loss_lambda_c =  361510764544.0
loss_lambda_c =  358703693824.0
loss_lambda_c =  356194189312.0
loss_lambda_c =  353945321472.0
loss_lambda_c =  351929892864.0
loss_lambda_c =  350125228032.0
loss_lambda_c =  348509372416.0
loss_lambda_c =  347061354496.0
loss_lambda_c =  345759711232.0
loss_lambda_c =  344585175040.0
loss_lambda_c =  343519232000.0
loss_lambda_c =  342545399808.0
loss_lambda_c =  341650309120.0
loss_lambda_c =  340822392832.0
loss_lambda_c =  340052836352.0
loss_lambda_c =  339334496256.0
loss_lambda_c =  338662195200.0
loss_lambda_c =  338031804416.0
loss_lambda_c =  3374400

loss_lambda_c =  925712152264704.0
loss_lambda_c =  30223475146752.0
loss_lambda_c =  7318841851904.0
loss_lambda_c =  4420921720832.0
loss_lambda_c =  3408171892736.0
loss_lambda_c =  2892631638016.0
loss_lambda_c =  2578068013056.0
loss_lambda_c =  2366744297472.0
loss_lambda_c =  2212946771968.0
loss_lambda_c =  2093711622144.0
loss_lambda_c =  1997346832384.0
loss_lambda_c =  1917687300096.0
loss_lambda_c =  1851227373568.0
loss_lambda_c =  1795720216576.0
loss_lambda_c =  1749485879296.0
loss_lambda_c =  1711120711680.0
loss_lambda_c =  1679374680064.0
loss_lambda_c =  1653124628480.0
loss_lambda_c =  1631374016512.0
loss_lambda_c =  1613271007232.0
loss_lambda_c =  1598082383872.0
loss_lambda_c =  1585231953920.0
loss_lambda_c =  1574266339328.0
loss_lambda_c =  1564847243264.0
loss_lambda_c =  1556718419968.0
loss_lambda_c =  1549694533632.0
loss_lambda_c =  1543633240064.0
loss_lambda_c =  1538422472704.0
loss_lambda_c =  1533967728640.0
loss_lambda_c =  1530184859648.0
loss_la

In [ ]:
model.fit(n_steps=50, with_precisions=False, z_iterations=200, thetac_iterations=10000, lambdac_iterations=500)

In [ ]:
model.fit(n_steps=50, z_iterations=100, with_precisions=True, lambdac_iterations=1000)

In [ ]:
model.log_lambdac_mean

In [ ]:
# model = gs.GenerativeSurrogate()
# model.load()

In [ ]:
test_samples = {n for n in range(0, 4)}
testData = dta.StokesData(unsupervised_samples=test_samples)
testData.read_data()
# trainingData.plotMicrostruct(1)
testData.reshape_microstructure_image()

In [ ]:
uf_pred, Z = model.predict(testData, max_iterations=1000)

In [ ]:
uf_pred[0]

In [ ]:
model.fit(n_steps=10)

In [ ]:
x = np.outer(np.linspace(0, 1, 129), np.ones(129))
y = x.copy().T # transpose

In [ ]:
fig = plt.figure()

In [ ]:
ax = plt.axes(projection='3d')

In [ ]:
ax.plot_surface(x, y, np.reshape(uf_pred[0], (129, 129)),cmap='viridis', edgecolor='none')

In [ ]:
pp = np.reshape(model.data.P[0, :], (129, 129))

In [ ]:
ax.plot_surface(x, y, pp.detach().numpy())

In [ ]:
model.plot_input_reconstruction()

In [ ]:
model.plot_generated_microstructures()

In [ ]:
model.lambdacOpt.param_groups[0]['lr']

In [ ]:
model.data.P[-1, :]

In [ ]:
model.rom.interpolated_solution.array

In [ ]:
model.data.P[-1, :]

In [ ]:
model.log_lambdac_mean

In [ ]:
model.taucf

In [ ]:
uf_pred = model.rom_autograd(torch.exp(model.log_lambdac_mean[0, :]) + 1e-12)

In [ ]:
uf_pred

In [ ]:
model.lambdacOpt.param_groups

In [ ]:
x = model.log_lambdac_mean.split(1)

In [ ]:
x[0]

In [ ]:
model.log_lambdac_mean

In [ ]:
from torch import optim
x = optim.Adam([])

In [ ]:
x = gs.LogLambdacParam()

In [ ]:
x.loss(5)

In [ ]:
x

In [ ]:
lst = []

In [ ]:
lst.append(gs.LogLambdacParam())

In [ ]:
x = 5
isinstance(x, int)